In [1]:
from pgmpy.models import NaiveBayes
from pgmpy.inference import VariableElimination
import pandas as pd
import numpy as np

In [2]:
model = NaiveBayes()

In [3]:
df = pd.read_csv("../data/very-small-SC-clean.csv", 
                         usecols=["id", "driver_gender", "driver_race", "officer_race", "stop_outcome"], 
                         dtype={"id" : "str", "driver_gender" : "category", "driver_race" : "category",
                                "officer_race" : "category", "stop_outcome" : "category"})

In [4]:
model.add_nodes_from(["driver_gender", "stop_outcome"])

In [5]:
model.add_edges_from([("stop_outcome", "driver_gender"), ("stop_outcome", "driver_race"), ("stop_outcome", "officer_race")])

In [6]:
df.drop(["id"], axis=1, inplace=True)

In [7]:
model.fit(df)

In [8]:
for cpd in model.get_cpds():
    print(cpd)

╒══════════════════╤══════════════════════╤════════════════════════╤═══════════════════════╕
│ stop_outcome     │ stop_outcome(Arrest) │ stop_outcome(Citation) │ stop_outcome(Warning) │
├──────────────────┼──────────────────────┼────────────────────────┼───────────────────────┤
│ driver_gender(F) │ 0.0                  │ 0.32857142857142857    │ 0.3076923076923077    │
├──────────────────┼──────────────────────┼────────────────────────┼───────────────────────┤
│ driver_gender(M) │ 1.0                  │ 0.6714285714285714     │ 0.6923076923076923    │
╘══════════════════╧══════════════════════╧════════════════════════╧═══════════════════════╛
╒═══════════════════════╤══════════════════════╤════════════════════════╤═══════════════════════╕
│ stop_outcome          │ stop_outcome(Arrest) │ stop_outcome(Citation) │ stop_outcome(Warning) │
├───────────────────────┼──────────────────────┼────────────────────────┼───────────────────────┤
│ driver_race(Black)    │ 0.6666666666666666   │ 0.2   

In [9]:
predict_data = df.copy()

In [10]:
predict_data.drop(["stop_outcome"], axis=1, inplace=True)

In [19]:
infer = VariableElimination(model)
r = infer.query(variables=["stop_outcome"], evidence={"driver_gender" : 1, "driver_race" : 0})
print(r['stop_outcome'])

╒════════════════╤═════════════════════╕
│ stop_outcome   │   phi(stop_outcome) │
╞════════════════╪═════════════════════╡
│ stop_outcome_0 │              0.1181 │
├────────────────┼─────────────────────┤
│ stop_outcome_1 │              0.5550 │
├────────────────┼─────────────────────┤
│ stop_outcome_2 │              0.3270 │
╘════════════════╧═════════════════════╛
